In [171]:
# 道路面积比例
# 居住面积比例
# 工业面积比例
# 建筑物占地面积的比例
# 在25米缓冲区范围内的平均建筑距离
# 平均POI密度

In [172]:
import geopandas as gpd
import pandas as pd
import glob

In [173]:
# 设置文件夹路径
folder_path = r'E:\work\spatio_evo_urbanvisenv_svi\2023拉萨市POI\西藏自治区-拉萨市-53123'  # 替换为你的文件夹路径

# 使用glob.glob查找所有的.shp文件
shapefiles = glob.glob(folder_path + '/*.shp')

# 初始化一个空的GeoDataFrame
gdf_combined = gpd.GeoDataFrame()

# 遍历所有找到的.shp文件
for shapefile in shapefiles:
    gdf = gpd.read_file(shapefile)  # 读取.shp文件
    gdf_combined = pd.concat([gdf_combined, gdf], ignore_index=True)  # 合并GeoDataFrame

# 确保合并后的GeoDataFrame的CRS是一致的
points_gdf = gdf_combined.set_crs(gdf.crs)

In [174]:
print(points_gdf.shape)
print(points_gdf.columns)

(53123, 25)
Index(['id', 'name', 'type', 'typecode', 'address', 'location', 'tel', 'pcode',
       'pname', 'citycode', 'cityname', 'adcode', 'adname', 'gridcode',
       'distance', 'business_a', 'timestamp', 'biz_ext', '行业大', '行业中', '行业小',
       'wgs84_经', 'wgs84_纬', 'statement', 'geometry'],
      dtype='object')


In [175]:
points_gdf_01 = gpd.GeoDataFrame()
for i in ['id','geometry']:
    # print(i[0])
    points_gdf_01[i] = points_gdf[i]
points_gdf_01.crs = "EPSG:4326"  

C:\Users\wang.tan.GOA\AppData\Local\Temp\ipykernel_1992\1828174798.py:4: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  points_gdf_01[i] = points_gdf[i]


In [176]:
print(points_gdf_01.columns)
print(points_gdf_01.head())

Index(['id', 'geometry'], dtype='object')
                                     id                   geometry
0  6e39378c-5301-3a54-8718-4b22e4e1cbb6  POINT (91.13029 29.65273)
1  494ca5ad-3172-3c1a-9a3e-004910b43e0c  POINT (91.04730 29.65995)
2  688482d2-badf-3398-b562-786b3586fe5a  POINT (91.12925 29.67620)
3  7ce9d7d2-851e-39da-8428-5700fc5edffb  POINT (91.13041 29.67678)
4  81a59424-c65a-3752-a25b-80d51b058aa5  POINT (91.10623 29.65841)


In [177]:
shapefile_path = r"e:\work\spatio_evo_urbanvisenv_svi\范围线\new_2024_03_25\urban_ten_indic_matrix_2013.shp"
polygons_gdf = gpd.read_file(shapefile_path)

In [178]:
# 计算每个多边形的面积
polygons_gdf['area'] = polygons_gdf.area
# 使用sjoin找到每个点所在的多边形
joined_gdf = gpd.sjoin(points_gdf_01, polygons_gdf, how='left', op='within')


C:\Users\wang.tan.GOA\AppData\Local\Temp\ipykernel_1992\2481880515.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygons_gdf['area'] = polygons_gdf.area
C:\Users\wang.tan.GOA\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [179]:
# 剔除任何含有NaN值的行
joined_gdf_clean = joined_gdf.dropna()
joined_gdf_clean.head()

,id,geometry,index_right,hexid,assortativ,area_m2,assortat_1,sky,plant,building,road,beautiful,boring,depressing,lively,safety,wealthy,sv_counts,area
0,6e39378c-5301-3a54-8718-4b22e4e1cbb6,POINT (91.13029 29.65273),1199.0,8a3c7274e85ffff,0.148148,19775.107154,0.148148,0.055447,0.004107,0.796098,0.088566,0.258672,0.283791,0.811679,0.696196,0.417794,0.340123,4.0,0.000001
3,7ce9d7d2-851e-39da-8428-5700fc5edffb,POINT (91.13041 29.67678),241.0,8a3c7274c92ffff,0.074074,19775.705108,0.074074,0.322176,0.078618,0.133862,0.355492,0.515447,0.478016,0.542766,0.883442,0.883331,0.501733,20.0,0.000001
11,29386ea2-2761-3731-83ae-8e6af6aa145c,POINT (91.13566 29.68170),1413.0,8a3c7276b51ffff,0.148148,19776.254849,0.148148,0.329367,0.015282,0.165790,0.522606,0.545741,0.418025,0.533085,0.978211,0.914074,0.633585,16.0,0.000001
13,bc65e934-77a5-33ae-9bdc-32ab899294ad,POINT (91.11615 29.64443),1027.0,8a3c7274ed97fff,0.037037,19773.747042,0.037037,0.261865,0.042740,0.229618,0.453190,0.549009,0.373898,0.526313,0.990575,0.958622,0.610901,12.0,0.000001
16,2485e9ef-9757-3a15-8fd9-f92308c8f934,POINT (91.13038 29.68320),682.0,8a3c7276a267fff,0.037037,19775.882879,0.037037,0.434569,0.016911,0.087604,0.505034,0.519700,0.572524,0.421886,0.971956,0.945395,0.496984,8.0,0.000001


In [180]:
# 计算每个多边形中的点数
point_counts = joined_gdf.groupby('hexid').size()
len(point_counts)
point_counts

hexid
8a3c7229604ffff    1
8a3c722960c7fff    1
8a3c722960cffff    2
8a3c722963a7fff    4
8a3c7229641ffff    6
                  ..
8a3c72773bb7fff    1
8a3c72773c4ffff    9
8a3c72773c67fff    2
8a3c72773c6ffff    3
8a3c72773d27fff    3
Length: 1383, dtype: int64

In [181]:
target_crs = 'EPSG:3395'
# 转换CRS
polygons_gdf_01 = polygons_gdf.to_crs(target_crs)
# 计算转换后的面积
polygons_gdf_01['area_m2'] = polygons_gdf_01.area
target_crs = 'EPSG:4326'
# 转换CRS
polygons_gdf_01 = polygons_gdf_01.to_crs(target_crs)

In [182]:
polygons_gdf_01.head()

,hexid,assortativ,area_m2,assortat_1,sky,plant,building,road,beautiful,boring,depressing,lively,safety,wealthy,sv_counts,geometry,area
0,8a3c72772727fff,0.037037,19764.780841,0.037037,0.403984,0.183773,0.165928,0.315303,0.554196,0.472688,0.490049,0.892701,0.942404,0.500882,20,"POLYGON ((91.01385 29.63066, 91.01312 29.63035...",0.000001
1,8a3c7274b927fff,0.037037,19777.962373,0.037037,0.370537,0.079257,0.131826,0.394364,0.593416,0.374476,0.585557,0.959126,0.936719,0.468131,16,"POLYGON ((91.16697 29.64801, 91.16625 29.64769...",0.000001
2,8a3c72746c27fff,0.185185,19768.449836,0.185185,0.181642,0.060681,0.185049,0.253272,0.521265,0.320144,0.619440,0.745786,0.765138,0.422366,20,"POLYGON ((91.05224 29.64873, 91.05152 29.64841...",0.000001
3,8a3c7274e0f7fff,0.000000,19775.011362,0.000000,0.045775,0.017328,0.126618,0.335060,0.469151,0.250189,0.722135,0.853497,0.812147,0.374623,4,"POLYGON ((91.13237 29.64385, 91.13165 29.64354...",0.000001
4,8a3c7274d09ffff,0.000000,19778.072726,0.000000,0.322169,0.010103,0.285851,0.567989,0.696102,0.626196,0.502875,0.990757,0.963775,0.623218,4,"POLYGON ((91.16221 29.66907, 91.16148 29.66875...",0.000001


In [183]:

point_counts_dadaframe = pd.DataFrame({'hexid': point_counts.index, 'poi_count': point_counts.values})
polygong_poi_df = polygons_gdf_01.merge(point_counts_dadaframe, on = 'hexid', how='left').fillna(0)

# print(polygong_poi_df)

In [184]:
polygong_poi_df.head(3)

,hexid,assortativ,area_m2,assortat_1,sky,plant,building,road,beautiful,boring,depressing,lively,safety,wealthy,sv_counts,geometry,area,poi_count
0,8a3c72772727fff,0.037037,19764.780841,0.037037,0.403984,0.183773,0.165928,0.315303,0.554196,0.472688,0.490049,0.892701,0.942404,0.500882,20,"POLYGON ((91.01385 29.63066, 91.01312 29.63035...",0.000001,0.0
1,8a3c7274b927fff,0.037037,19777.962373,0.037037,0.370537,0.079257,0.131826,0.394364,0.593416,0.374476,0.585557,0.959126,0.936719,0.468131,16,"POLYGON ((91.16697 29.64801, 91.16625 29.64769...",0.000001,5.0
2,8a3c72746c27fff,0.185185,19768.449836,0.185185,0.181642,0.060681,0.185049,0.253272,0.521265,0.320144,0.619440,0.745786,0.765138,0.422366,20,"POLYGON ((91.05224 29.64873, 91.05152 29.64841...",0.000001,0.0


In [185]:
# 将点数除以多边形的面积，得到密度
polygong_poi_df['poi_density'] = polygong_poi_df['poi_count']*1000 / polygons_gdf_01['area_m2']
polygong_poi_df.head(1)

,hexid,assortativ,area_m2,assortat_1,sky,plant,building,road,beautiful,boring,depressing,lively,safety,wealthy,sv_counts,geometry,area,poi_count,poi_density
0,8a3c72772727fff,0.037037,19764.780841,0.037037,0.403984,0.183773,0.165928,0.315303,0.554196,0.472688,0.490049,0.892701,0.942404,0.500882,20,"POLYGON ((91.01385 29.63066, 91.01312 29.63035...",0.000001,0.0,0.0


In [186]:
# 删除area_m2列
polygong_poi_df_01 = polygong_poi_df.drop(['assortat_1','poi_count','area','area_m2'], axis=1)
polygong_poi_df_01.head(1)

,hexid,assortativ,sky,plant,building,road,beautiful,boring,depressing,lively,safety,wealthy,sv_counts,geometry,poi_density
0,8a3c72772727fff,0.037037,0.403984,0.183773,0.165928,0.315303,0.554196,0.472688,0.490049,0.892701,0.942404,0.500882,20,"POLYGON ((91.01385 29.63066, 91.01312 29.63035...",0.0


In [187]:
shapefile_path = r"e:\work\spatio_evo_urbanvisenv_svi\2018拉萨市土地利用\拉萨市.shp"
land_use_gdf = gpd.read_file(shapefile_path)
land_use_gdf_01 = gpd.GeoDataFrame()
for i in ['Level1_cn','geometry']:
    # print(i[0])
    land_use_gdf_01[i] = land_use_gdf[i]
land_use_gdf_01.crs = "EPSG:4326"  
land_use_gdf_01[0:3]

C:\Users\wang.tan.GOA\AppData\Local\Temp\ipykernel_1992\4208166305.py:6: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  land_use_gdf_01[i] = land_use_gdf[i]


,Level1_cn,geometry
0,商业用地,"POLYGON ((90.73966 29.35263, 90.73650 29.35185..."
1,公共管理和服务用地,"POLYGON ((90.74584 29.35415, 90.74235 29.35330..."
2,居住用地,"POLYGON ((90.74637 29.35281, 90.74630 29.35281..."


In [188]:
shapefile_path = r"e:\work\spatio_evo_urbanvisenv_svi\拉萨_建筑轮廓\lasa.shp"
building_outline_gdf = gpd.read_file(shapefile_path)
building_outline_gdf_01 = gpd.GeoDataFrame()
for i in ['geometry']:
    # print(i[0])
    building_outline_gdf_01[i] = building_outline_gdf[i]
building_outline_gdf_01.crs = "EPSG:4326"  
building_outline_gdf_01[0:3]

C:\Users\wang.tan.GOA\AppData\Local\Temp\ipykernel_1992\2999638884.py:6: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  building_outline_gdf_01[i] = building_outline_gdf[i]


,geometry
0,"POLYGON ((91.03125 29.65136, 91.03125 29.65117..."
1,"POLYGON ((91.03125 29.65152, 91.03125 29.65142..."
2,"POLYGON ((91.03085 29.64619, 91.03125 29.64616..."


In [189]:
building_outline_gdf.head(1)

,Id,Floor,geometry
0,0,2,"POLYGON ((91.03125 29.65136, 91.03125 29.65117..."


In [190]:
# 在25米缓冲区范围内的平均建筑距离

target_crs = 'EPSG:32633'
# 转换CRS
building_outline_gdf_02 = building_outline_gdf.to_crs(target_crs)
building_outline_gdf_02.head(1)

,Id,Floor,geometry
0,0,2,"POLYGON ((8354600.805 7452564.824, 8354635.107..."


In [191]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import nearest_points
import numpy as np

# 创建一个新列，包含id_0, id_1, ... 格式的字符串
building_outline_gdf_02['id_'] = ['id_' + str(i) for i in range(building_outline_gdf_02.shape[0])]

# 计算每个多边形的中心点
building_outline_gdf_02['center'] = building_outline_gdf_02.geometry.centroid

# 以每个多边形的中心点为基准点，以25米为半径绘制圆
building_outline_gdf_02['circle'] = building_outline_gdf_02['center'].buffer(25)

building_outline_gdf_circle = gpd.GeoDataFrame()
building_outline_gdf_circle['geometry'] = building_outline_gdf_02['circle'] 
building_outline_gdf_circle['id_'] = building_outline_gdf_02['id_'] 
building_outline_gdf_circle['geometry_0'] = building_outline_gdf_02['geometry']

building_outline_gdf_circle.crs = "EPSG:32633"  


C:\Users\wang.tan.GOA\AppData\Local\Temp\ipykernel_1992\1156939680.py:16: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  building_outline_gdf_circle['geometry'] = building_outline_gdf_02['circle']


In [192]:
building_outline_gdf_02.shape
building_outline_gdf_02.head(1)

,Id,Floor,geometry,id_,center,circle
0,0,2,"POLYGON ((8354600.805 7452564.824, 8354635.107...",id_0,POINT (8354616.740 7452536.094),"POLYGON ((8354641.740 7452536.094, 8354641.620..."


In [193]:
building_outline_gdf_circle.head(1)

,geometry,id_,geometry_0
0,"POLYGON ((8354641.740 7452536.094, 8354641.620...",id_0,"POLYGON ((8354600.805 7452564.824, 8354635.107..."


In [194]:
intersection_gdf_cir = gpd.overlay(building_outline_gdf_circle, building_outline_gdf_02, how='intersection')

In [195]:
intersection_gdf_cir.head(1)
# intersection_gdf_cir.shape
# building_outline_gdf_circle.shape
# building_outline_gdf_02.shape

,id__1,geometry_0,Id,Floor,id__2,center,circle,geometry
0,id_0,"POLYGON ((8354600.805 7452564.824, 8354635.107...",0,2,id_0,POINT (8354616.740 7452536.094),"POLYGON ((8354641.740 7452536.094, 8354641.620...","POLYGON ((8354634.418 7452518.416, 8354632.600..."


In [196]:
# 初始化一个列表来存储距离
distances = []

# 遍历每行数据
for index, row in intersection_gdf_cir.iterrows():
    # 获取当前行的point和polygon
    polygon01 = row['geometry_0']
    polygon02 = row['geometry']
    
    # 使用nearest_points计算最近点
    nearest = nearest_points(polygon01, polygon02)
    
    # 计算两点之间的距离
    distance = nearest[0].distance(nearest[1])
    
    # 将距离添加到列表中
    distances.append(distance)

# 将计算出的距离添加到GeoDataFrame中
intersection_gdf_cir['nearest_distance'] = distances



In [197]:
# 查看结果
intersection_gdf_cir.head(5)

,id__1,geometry_0,Id,Floor,id__2,center,circle,geometry,nearest_distance
0,id_0,"POLYGON ((8354600.805 7452564.824, 8354635.107...",0,2,id_0,POINT (8354616.740 7452536.094),"POLYGON ((8354641.740 7452536.094, 8354641.620...","POLYGON ((8354634.418 7452518.416, 8354632.600...",0.000000
1,id_35,"POLYGON ((8354600.805 7452564.824, 8354601.669...",0,2,id_0,POINT (8354616.740 7452536.094),"POLYGON ((8354641.740 7452536.094, 8354641.620...","POLYGON ((8354634.848 7452544.278, 8354632.744...",0.000000
2,id_46,"POLYGON ((8354569.933 7452580.156, 8354569.938...",0,2,id_0,POINT (8354616.740 7452536.094),"POLYGON ((8354641.740 7452536.094, 8354641.620...","POLYGON ((8354600.797 7452564.395, 8354600.805...",9.759653
3,id_0,"POLYGON ((8354600.805 7452564.824, 8354635.107...",0,2,id_35,POINT (8354620.959 7452565.065),"POLYGON ((8354645.959 7452565.065, 8354645.839...","POLYGON ((8354611.863 7452560.613, 8354614.290...",0.000000
4,id_35,"POLYGON ((8354600.805 7452564.824, 8354601.669...",0,2,id_35,POINT (8354620.959 7452565.065),"POLYGON ((8354645.959 7452565.065, 8354645.839...","POLYGON ((8354601.669 7452575.110, 8354635.982...",0.000000


In [198]:
# 计算每个多边形中的点数
building_outline_gdf_25 = intersection_gdf_cir.groupby('id__2')['nearest_distance'].mean()
building_outline_gdf_25 = pd.DataFrame({'id_': building_outline_gdf_25.index, 'nearest_distance': building_outline_gdf_25.values})
building_outline_gdf_03 = building_outline_gdf_02.merge(building_outline_gdf_25, on = 'id_', how='left')
# 在25米缓冲区范围内的平均建筑距离

target_crs = 'EPSG:4326'
# 转换CRS
building_outline_gdf_04 = building_outline_gdf_03.to_crs(target_crs)

In [199]:
building_outline_gdf_03.head(1)

,Id,Floor,geometry,id_,center,circle,nearest_distance
0,0,2,"POLYGON ((8354600.805 7452564.824, 8354635.107...",id_0,POINT (8354616.740 7452536.094),"POLYGON ((8354641.740 7452536.094, 8354641.620...",3.253218


In [200]:
shapefile_path = r"e:\work\spatio_evo_urbanvisenv_svi\lhasa_road_dissovle\lhasa_road_dissovle.shp"
road_outline_gdf = gpd.read_file(shapefile_path)
road_outline_gdf_01 = gpd.GeoDataFrame()
for i in ['geometry']:
    # print(i[0])
    road_outline_gdf_01[i] = road_outline_gdf[i]
road_outline_gdf_01.crs = "EPSG:4326"  
road_outline_gdf_01[0:3]

C:\Users\wang.tan.GOA\AppData\Local\Temp\ipykernel_1992\2450535243.py:6: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  road_outline_gdf_01[i] = road_outline_gdf[i]


,geometry
0,"POLYGON ((90.49421 29.24828, 90.49416 29.24828..."
1,"POLYGON ((90.46670 29.25600, 90.46754 29.25468..."
2,"POLYGON ((90.47103 29.25643, 90.47105 29.25640..."


In [201]:
# 获取hexid列的唯一值
unique_hexids = land_use_gdf_01['Level1_cn'].unique()
print(unique_hexids)

['商业用地' '公共管理和服务用地' '居住用地' '工业用地']


In [202]:
# 使用loc函数选择Level1_cn列中值为“工业用地”的行
land_use_gdf_industrial = land_use_gdf_01.loc[land_use_gdf_01['Level1_cn'] == "工业用地"]
# 使用loc函数选择Level1_cn列中值为“工业用地”的行
land_use_gdf_residential = land_use_gdf_01.loc[land_use_gdf_01['Level1_cn'] == "居住用地"]

In [203]:
import geopandas as gpd
from shapely.geometry import Polygon

intersection_gdf_residential = gpd.overlay(polygong_poi_df_01, land_use_gdf_residential, how='intersection')
ratios = intersection_gdf_residential.area / polygong_poi_df_01.area
polygong_poi_df_01['residential_ratio'] = ratios

intersection_gdf_industrial = gpd.overlay(polygong_poi_df_01, land_use_gdf_industrial, how='intersection')
ratios = intersection_gdf_industrial.area / polygong_poi_df_01.area
polygong_poi_df_01['industrial_ratio'] = ratios

intersection_gdf_building = gpd.overlay(polygong_poi_df_01, building_outline_gdf_04, how='intersection')
ratios = intersection_gdf_building.area / polygong_poi_df_01.area
polygong_poi_df_01['building_ratio'] = ratios

intersection_gdf_road = gpd.overlay(polygong_poi_df_01, road_outline_gdf_01, how='intersection')
ratios = intersection_gdf_road.area / polygong_poi_df_01.area
polygong_poi_df_01['road_ratio'] = ratios


C:\Users\wang.tan.GOA\AppData\Local\Temp\ipykernel_1992\3770274531.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ratios = intersection_gdf_residential.area / polygong_poi_df_01.area
C:\Users\wang.tan.GOA\AppData\Local\Temp\ipykernel_1992\3770274531.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ratios = intersection_gdf_industrial.area / polygong_poi_df_01.area
C:\Users\wang.tan.GOA\AppData\Local\Temp\ipykernel_1992\3770274531.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ratios = intersection_gdf_building.area / polygong_poi_df_01.area
C:\Users\wang.tan.GOA\AppDat

In [204]:
intersection_gdf_building_25 = gpd.overlay(polygong_poi_df_01, building_outline_gdf_04, how='intersection')
intersection_gdf_building_25.head(1)


,hexid,assortativ,sky,plant,building,road,beautiful,boring,depressing,lively,...,industrial_ratio,building_ratio,road_ratio,Id,Floor,id_,center,circle,nearest_distance,geometry
0,8a3c72746c27fff,0.185185,0.181642,0.060681,0.185049,0.253272,0.521265,0.320144,0.61944,0.745786,...,0.729511,0.083801,0.129216,0,2,id_2069,POINT (8356647.328 7455748.770),"POLYGON ((8356672.328 7455748.770, 8356672.208...",0.0,"POLYGON ((91.05228 29.64926, 91.05211 29.64926..."


In [205]:
# 计算每个多边形中的点数
intersection_gdf_building_25 = intersection_gdf_building_25.groupby('hexid')['nearest_distance'].mean()
intersection_gdf_building_25 = pd.DataFrame({'hexid': intersection_gdf_building_25.index, 'nearest_distance': intersection_gdf_building_25.values})
polygong_poi_df_02 = polygong_poi_df_01.merge(intersection_gdf_building_25, on = 'hexid', how='left').fillna(0)

In [206]:
polygong_poi_df_02.head(3)
# polygong_poi_df_01.shape
# polygong_poi_df_02 = polygong_poi_df_01.fillna(0)
# polygong_poi_df_02


,hexid,assortativ,sky,plant,building,road,beautiful,boring,depressing,lively,safety,wealthy,sv_counts,geometry,poi_density,residential_ratio,industrial_ratio,building_ratio,road_ratio,nearest_distance
0,8a3c72772727fff,0.037037,0.403984,0.183773,0.165928,0.315303,0.554196,0.472688,0.490049,0.892701,0.942404,0.500882,20,"POLYGON ((91.01385 29.63066, 91.01312 29.63035...",0.000000,0.242798,0.000593,0.016093,0.090968,0.000000
1,8a3c7274b927fff,0.037037,0.370537,0.079257,0.131826,0.394364,0.593416,0.374476,0.585557,0.959126,0.936719,0.468131,16,"POLYGON ((91.16697 29.64801, 91.16625 29.64769...",0.252807,0.999850,0.039908,0.003957,0.119992,0.000000
2,8a3c72746c27fff,0.185185,0.181642,0.060681,0.185049,0.253272,0.521265,0.320144,0.619440,0.745786,0.765138,0.422366,20,"POLYGON ((91.05224 29.64873, 91.05152 29.64841...",0.000000,1.000340,0.729511,0.083801,0.129216,0.993648


In [207]:
sub_polygong_poi_df_02 = polygong_poi_df_02.iloc[:,2:12]
sub_polygong_poi_df_02.head()

,sky,plant,building,road,beautiful,boring,depressing,lively,safety,wealthy
0,0.403984,0.183773,0.165928,0.315303,0.554196,0.472688,0.490049,0.892701,0.942404,0.500882
1,0.370537,0.079257,0.131826,0.394364,0.593416,0.374476,0.585557,0.959126,0.936719,0.468131
2,0.181642,0.060681,0.185049,0.253272,0.521265,0.320144,0.619440,0.745786,0.765138,0.422366
3,0.045775,0.017328,0.126618,0.335060,0.469151,0.250189,0.722135,0.853497,0.812147,0.374623
4,0.322169,0.010103,0.285851,0.567989,0.696102,0.626196,0.502875,0.990757,0.963775,0.623218


In [208]:
import geopandas as gpd
from sklearn.cluster import KMeans
import numpy as np
# 使用KMeans进行聚类分析
kmeans = KMeans(n_clusters=6, random_state=0).fit(sub_polygong_poi_df_02)
# 获取聚类标签
labels = kmeans.labels_
labels.shape 

C:\Users\wang.tan.GOA\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


(1866,)

In [209]:
polygong_poi_df_02['cluster'] = labels
polygong_poi_df_02.head(3)

,hexid,assortativ,sky,plant,building,road,beautiful,boring,depressing,lively,...,wealthy,sv_counts,geometry,poi_density,residential_ratio,industrial_ratio,building_ratio,road_ratio,nearest_distance,cluster
0,8a3c72772727fff,0.037037,0.403984,0.183773,0.165928,0.315303,0.554196,0.472688,0.490049,0.892701,...,0.500882,20,"POLYGON ((91.01385 29.63066, 91.01312 29.63035...",0.000000,0.242798,0.000593,0.016093,0.090968,0.000000,2
1,8a3c7274b927fff,0.037037,0.370537,0.079257,0.131826,0.394364,0.593416,0.374476,0.585557,0.959126,...,0.468131,16,"POLYGON ((91.16697 29.64801, 91.16625 29.64769...",0.252807,0.999850,0.039908,0.003957,0.119992,0.000000,5
2,8a3c72746c27fff,0.185185,0.181642,0.060681,0.185049,0.253272,0.521265,0.320144,0.619440,0.745786,...,0.422366,20,"POLYGON ((91.05224 29.64873, 91.05152 29.64841...",0.000000,1.000340,0.729511,0.083801,0.129216,0.993648,1


In [210]:
output_path = r"E:\work\spatio_evo_urbanvisenv_svi\范围线\new_2024_03_25\six_urban_16_cluser_2013.shp"
polygong_poi_df_02.to_file(output_path, driver='ESRI Shapefile')

C:\Users\wang.tan.GOA\AppData\Local\Temp\ipykernel_1992\3682642763.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  polygong_poi_df_02.to_file(output_path, driver='ESRI Shapefile')
